In [8]:
import numpy as np

import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro
import scipy.stats as stats
import scipy.stats as ss

In [2]:
ab = pd.read_excel("df_ab_test.xlsx")

In [3]:
ab

,data,sample
0,101.622050,test
1,97.693698,test
2,108.837310,test
3,88.519357,test
4,91.358254,test
...,...,...
795,100.974802,control
796,96.527106,control
797,109.217752,control
798,99.092552,control


In [4]:
# Summary Stats: sum_gamerounds
ab.describe([0.01, 0.05, 0.10, 0.20, 0.80, 0.90, 0.95, 0.99])[["data"]].T
#.T значения будут в строку

,count,mean,std,min,1%,5%,10%,20%,50%,80%,90%,95%,99%,max
data,800.0,100.050762,5.312902,80.791472,87.406499,91.395161,93.216322,95.45792,99.994645,104.783756,106.701855,108.844512,111.153999,115.008118


In [5]:
# A/B Groups & Target Summary Stats
ab.groupby("sample").data.agg(["count", "median", "mean", "std", "max"])

,count,median,mean,std,max
sample,,,,,
control,400,100.131783,100.200502,5.158982,112.435521
test,400,99.694387,99.901022,5.464846,115.008118


In [6]:
ab["sample"] = np.where(ab['sample'] == "test", "A", "B")
ab

,data,sample
0,101.622050,A
1,97.693698,A
2,108.837310,A
3,88.519357,A
4,91.358254,A
...,...,...
795,100.974802,B
796,96.527106,B
797,109.217752,B
798,99.092552,B


In [7]:
# A/B Testing Function - Quick Solution
# target изучаемая метрика
def AB_Test(dataframe, group, target):
    
    # Packages
    from scipy.stats import shapiro
    import scipy.stats as stats
    
    # Split A/B
    groupA = dataframe[dataframe[group] == "A"][target]
    groupB = dataframe[dataframe[group] == "B"][target]
    
    # Assumption: Normality
    ntA = shapiro(groupA)[1] < 0.05
    ntB = shapiro(groupB)[1] < 0.05
    # H0: Distribution is Normal! - False
    # H1: Distribution is not Normal! - True
    
    if (ntA == False) & (ntB == False): # "H0: Normal Distribution"
        # Parametric Test
        # Assumption: Homogeneity of variances
        leveneTest = stats.levene(groupA, groupB)[1] < 0.05
        # H0: Homogeneity: False
        # H1: Heterogeneous: True
        
        if leveneTest == False:
            # Homogeneity
            ttest = stats.ttest_ind(groupA, groupB, equal_var=True)[1]
            # H0: M1 == M2 - False
            # H1: M1 != M2 - True
        else:
            # Heterogeneous
            ttest = stats.ttest_ind(groupA, groupB, equal_var=False)[1]
            # H0: M1 == M2 - False
            # H1: M1 != M2 - True
    else:
        # Non-Parametric Test
        ttest = stats.mannwhitneyu(groupA, groupB)[1] 
        # H0: M1 == M2 - False
        # H1: M1 != M2 - True
        
    # Result
    temp = pd.DataFrame({
        "AB Hypothesis":[ttest < 0.05], 
        "p-value":[ttest]
    })
    temp["Test Type"] = np.where((ntA == False) & (ntB == False), "Parametric", "Non-Parametric")
    temp["AB Hypothesis"] = np.where(temp["AB Hypothesis"] == False, "Fail to Reject H0", "Reject H0")
    temp["Comment"] = np.where(temp["AB Hypothesis"] == "Fail to Reject H0", "A/B groups are similar!", "A/B groups are not similar!")
    
    # Columns
    if (ntA == False) & (ntB == False):
        temp["Homogeneity"] = np.where(leveneTest == False, "Yes", "No")
        temp = temp[["Test Type", "Homogeneity","AB Hypothesis", "p-value", "Comment"]]
    else:
        temp = temp[["Test Type","AB Hypothesis", "p-value", "Comment"]]
    
    # Print Hypothesis
    print("# A/B Testing Hypothesis")
    print("H0: A == B")
    print("H1: A != B", "\n")
    
    return temp
    
    
    
# Apply A/B Testing
AB_Test(dataframe=ab, group = "sample", target = "data")


# A/B Testing Hypothesis
H0: A == B
H1: A != B 



,Test Type,Homogeneity,AB Hypothesis,p-value,Comment
0,Parametric,Yes,Fail to Reject H0,0.425695,A/B groups are similar!


In [13]:
test = ab[ab['sample'] == "A"]['data']
control = ab[ab['sample'] == "B"]['data']

<span style="font-weight: bold;">Тестируем на нормальность</span>

In [15]:
#через scipy
 
print(ss.shapiro(test))
print(ss.shapiro(control))

ShapiroResult(statistic=0.9957486495220561, pvalue=0.35406769376096614)
ShapiroResult(statistic=0.9935246435614979, pvalue=0.08447904959497456)


<span style="font-weight: bold;">Вывод</span> 

На основании результатов теста Шапиро-Уилка можно сделать вывод, что данные тестовой группы, вероятно, имеют нормальное распределение, так как p-значение (0.354) больше уровня значимости 0.05. Для данных из контрольной группы p-значение (0.084) также превышает уровень значимости, что также указывает на то, что эти данные вероятно имеют нормальное распределение.

<span style="font-weight: bold;">Проверяем на дисперсию с помощью критерия Левина</span>

In [16]:
ss.levene(test, control)

LeveneResult(statistic=1.8230727405150926, pvalue=0.17733076894977076)

<span style="font-weight: bold;">Вывод</span> 

Для результатов теста Левена (Levene test) значение p-value равно 0.177, что значительно превышает уровень значимости 0.05. Это означает, что у нас нет статистически значимых доказательств против нулевой гипотезы о равенстве дисперсий двух выборок. Следовательно, мы не можем отвергнуть нулевую гипотезу о равенстве дисперсий.

In [17]:
#t критерий стьюдента
ss.ttest_ind(test, control)

TtestResult(statistic=-0.7969885197879127, pvalue=0.42569475614907903, df=798.0)

<span style="font-weight: bold;">Вывод</span> 

На основании результатов t-теста можно сделать вывод, что нет статистически значимой разницы между средними значениями тестовой и контрольной группы, так как p-значение (0.426) значительно превышает уровень значимости 0.05. Это означает, что мы не можем отвергнуть нулевую гипотезу о равенстве средних значений двух выборок.